In [1]:
import os
from getpass import getpass

In [2]:
from utils import ChatOpenAI
course_api_key = getpass(prompt="Введите ваш ключ, полученный в боте курса")

llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

Введите ваш ключ, полученный в боте курса ········


In [11]:
!pip install GitPython langchain-chroma -qq

In [4]:
from git import Repo
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language

In [284]:
import os
import git
import csv

# Constants
REPO_URL = "https://github.com/nlmk-group/ds-2.0/"
CLONE_DIR = "cloned_repo"
COMPONENTS_DIR = os.path.join(CLONE_DIR, "src", "components")
OUTPUT_CSV = "combined_code.csv"
#TARGET_COMPONENTS = ["Alert", "Input", "AttachFiles"]
SPLIT_THRESHOLD = 7500  # Max number of symbols before splitting
OVERLAP_SIZE = 4000  # Overlap size for splitting
EXCLUDE = False

In [ ]:
# Function to clone the repository
def clone_repo():
    if not os.path.exists(CLONE_DIR):
        git.Repo.clone_from(REPO_URL, CLONE_DIR)
        print(f"Repository cloned to {CLONE_DIR}")
    else:
        print(f"Repository already cloned at {CLONE_DIR}")

clone_repo()

In [323]:
# Function to extract description and code from Stories.tsx
def extract_code_and_description(content):
    descriptions = []
    codes = []

    while 'description=' in content and 'code={' in content:
        # Extract description
        desc_start = content.find('description="') + len('description="')
        desc_end = content.find('"', desc_start)
        description = content[desc_start:desc_end]
        descriptions.append(description)
        
        # Extract code block with balanced braces
        code_start = content.find('code={') + len('code={')
        brace_count = 1
        code_end = code_start

        while brace_count > 0 and code_end < len(content):
            if content[code_end] == '{':
                brace_count += 1
            elif content[code_end] == '}':
                brace_count -= 1
            code_end += 1
        
        # Extract code content
        code_block = content[code_start:code_end - 1].strip()  # Exclude the final '}'
        codes.append(code_block)
        
        # Remove processed part from content
        content = content[code_end:]

    # Combine description and code, format description as comment
    result = []
    for desc, code in zip(descriptions, codes):
        result.append(f"// {desc}\n{code}")

    # Join all extracted parts with a double new line
    return "\n\n".join(result)

# Function to check if 'isStable' exists in Stories.tsx and gather formatted code
def get_stories_content(stories_path, component_name):
    if os.path.isfile(stories_path):
        with open(stories_path, "r") as stories_file:
            content = stories_file.read()
            if "isStable" in content:
                # Extract and format the code and description
                formatted_content = extract_code_and_description(content)
                if formatted_content:
                    # Format as component_name_how_to_use for Stories.tsx
                    relative_path = f"{component_name}_how_to_use"
                    return [f"{component_name} / {relative_path}", formatted_content]
    return []

# Function to fetch the full file content without splitting
def get_file_content(file_path, component_name, relative_path):
    with open(file_path, "r") as file:
        content = file.read()
        return [f"{component_name} / {relative_path}", content]

# Recursive function to fetch content from component files
def process_folder(folder_path, component_name):
    file_rows = []

    # Check the _stories folder for Stories.tsx and get its content if 'isStable' is present
    stories_path = os.path.join(folder_path, "_stories", "Stories.tsx")
    stories_content = get_stories_content(stories_path, component_name)
    
    # If no 'isStable' in Stories.tsx, skip this component
    if not stories_content:
        return []
    
    # Add content from Stories.tsx
    file_rows.append(stories_content)

    # Gather content from "types.ts", "enums.ts", and any ".d.ts" files
    for file_name in ["types.ts", "enums.ts"]:
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            relative_path = os.path.relpath(file_path, folder_path)
            file_rows.append(get_file_content(file_path, component_name, relative_path))
    
    # Look for any .d.ts files
    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith(".d.ts"):
                file_path = os.path.join(root, file_name)
                relative_path = os.path.relpath(file_path, folder_path)
                file_rows.append(get_file_content(file_path, component_name, relative_path))
    
    return file_rows

# Main function to combine code for all components
def combine_code_testing():
    all_rows = []
    
    # Process every component in the directory
    for folder_name in os.listdir(COMPONENTS_DIR):
        folder_path = os.path.join(COMPONENTS_DIR, folder_name)
        
        if os.path.isdir(folder_path):
            file_rows = process_folder(folder_path, folder_name)
            all_rows.extend(file_rows)  # Append the rows from each folder
    
    # Write the content into a CSV file
    with open(OUTPUT_CSV, "w", newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        
        # Write the header
        writer.writerow(["File", "Content"])
        
        # Write all the file rows (Component/relative path and content)
        writer.writerows(all_rows)
    
    print(f"Combined code for all components written to {OUTPUT_CSV}, with each component's content in a single row.")

# Start processing
combine_code_testing()

Combined code for all components written to combined_code.csv, with each component's content in a single row.


In [324]:
from langchain.document_loaders import CSVLoader, DataFrameLoader
loader = CSVLoader(file_path="./combined_code.csv")

documents = loader.load()

In [325]:
print([len(doc.page_content) for doc in documents])

[3586, 1117, 553, 866, 1843, 810, 334, 1108, 1272, 340, 194, 3839, 460, 237, 2292, 530, 344, 363, 896, 353, 416, 427, 4540, 268, 1657, 508, 329, 831, 7481, 306, 3980, 531, 561, 701, 170, 1956, 144, 348, 200, 345, 345, 320, 448, 353, 812, 272, 1409, 242, 1947, 145, 806, 1279, 880, 986, 580, 650, 136, 515, 232, 470, 135, 1170, 359, 138, 329, 2390, 1197, 3697, 563, 372, 199, 621, 977, 14115, 918, 301, 1609, 1293, 136, 3252, 308, 876, 606, 526, 5572, 457, 278, 4499, 861, 264, 1087, 732, 1545, 1030, 757, 347, 499, 2093, 1007, 619, 233, 677, 1494, 535, 396, 5713, 371, 1099, 1494]


In [334]:
documents[2]

Document(page_content="File: Button / enums.ts\nContent: /**\n * @type {enum} EButtonSizes\n * @type {enum} EButtonVariant\n * @type {enum} EButtonFill\n */\n\nexport enum EButtonSizes {\n  m = 'm',\n  s = 's',\n  xs = 'xs'\n}\n\nexport enum EButtonNodesPosition {\n  left = 'left',\n  right = 'right'\n}\n\nexport enum EButtonVariant {\n  primary = 'primary',\n  secondary = 'secondary',\n  grey = 'grey',\n  black = 'black',\n  success = 'success',\n  warning = 'warning',\n  error = 'error',\n  info = 'info'\n}\n\nexport enum EButtonFill {\n  solid = 'solid',\n  outline = 'outline',\n  clear = 'clear'\n}", metadata={'source': './combined_code.csv', 'row': 2})

In [183]:
from utils import OpenAIEmbeddings

embeddings_api_model = OpenAIEmbeddings(course_api_key=course_api_key)

In [327]:
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings(course_api_key=course_api_key)
db = FAISS.from_documents(
    documents, embeddings)

db.save_local("faiss_db_test")

In [328]:
retriever = db.as_retriever()

In [329]:
code_sample = """
            ```tsx
            // Import necessary components
            import React from 'react';
            import { ComponentName1, ComponentName2 } from '@nlmk/ds-2.0';
            
            const Interface = () => {
              return (
                <div>
                </div>
              );
            };
            
            // Export the main component
            export default Interface;"""

In [265]:
#code_sample = """
#            ```tsx\n// Import necessary components\nimport React from 'react';\nimport { ComponentName1, ComponentName2 } from '@nlmk/ds-2.0';\nconst Interface = () => {\nreturn (\n<div>\n</div>\n);\n};\n// Export the main component\nexport default Interface;"""

In [340]:
from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableMap

template = """
Generate typescript react code.
Use only nlmk components described below in the following context
usage examples have how_to_use postfix in index
in every enums.ts file you have values of properties for each component:
{context}

Question: {question}

The final code should follow the following structure:
{code_sample}

now very important: you can make imports only from nlmk.
do not forget to imoprt react
"""
prompt = ChatPromptTemplate.from_template(template)


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

chain = RunnableMap({
    "context": retriever | format_docs,  # Get context and format it
    "code_sample": RunnablePassthrough(),  # Pass the code sample through unchanged
    "question": RunnablePassthrough()  # Pass the question unchanged
}) | prompt | llm | StrOutputParser()

In [356]:
ans = chain.invoke('''
Мне нужен интерфейс для сервиса FrontendCopilot который является инструментом для автоматической генерации Веб Интерфейсов на type_script, базируясь на запросе пользователя. Интерфейс должен выглядеть следующим образом:
  - Слева во всю высоту страницу область, в которой Пользователь может видеть всю историю взаимодействия с системой. В этой области располагаются вертикально друг за другом кликабельные карточки со сжатым изображением результата прошлых генераций. Эту область пользователь может прокручивать, так как в во всю высоту должно помещаться ограниченное количество карточек - например 5.
  - По центру интерфейса большая прямоугольная область. В ней располагается окно вывода результата(сгенерированного интерфейса) - оно имеет два состояния: Рендер - отрендеренный результат работы сервиса, и Код - сгенерированный код интерфейса Сервисом показанный в удобном формате. Эти состояния переключаются нажатием на одну из табличек, прикрепленных к верхней левой части главной области.
  - Снизу под центральной областью - поле для ввода запроса пользователя. Оно должно вмещать несколько строк текста. Справа от этого поля - Кнопка отправки запроса.
  
  Пожалуйста сконфигурируй красивый интерфейс, чтобы им было удобно пользоваться: Вводить текстовый запрос - Отправлять Его нажатием кнопки - Иметь возможность удобно переключаться между рендером сгенерированного компонента и его исходным кодом с помощью переключателя - Видеть историю результатов обработки прошлых запросов на карточках в левой части. Все три области не должны быть приклеенными друг к другу и располагаться на небольшом одинаковом расстоянии. Покрась каждую область в приятные но разные цвета пожалуйста
''')
ans

'```tsx\nimport React from \'react\';\nimport { Card, Box, Tooltip, Input, Button } from \'@nlmk/ds-2.0\';\n\nconst FrontendCopilotInterface = () => {\n  return (\n    <div style={{ display: \'flex\', justifyContent: \'space-between\', padding: \'20px\' }}>\n      <div style={{ width: \'30%\', height: \'100vh\', overflowY: \'scroll\', backgroundColor: \'#f0f0f0\' }}>\n        <Card>\n          <Tooltip description="Результат генерации">\n            <img src="generated_result.jpg" alt="Generated Result" style={{ width: \'100%\', height: \'auto\' }} />\n          </Tooltip>\n        </Card>\n        <Card>\n          <Tooltip description="Результат генерации">\n            <img src="generated_result.jpg" alt="Generated Result" style={{ width: \'100%\', height: \'auto\' }} />\n          </Tooltip>\n        </Card>\n        <Card>\n          <Tooltip description="Результат генерации">\n            <img src="generated_result.jpg" alt="Generated Result" style={{ width: \'100%\', height: \'a

In [357]:
# Replace the \n symbols with actual newlines
real_ts_code = ans.replace("\\n", "\n")

# Optionally remove the '```tsx' and '```' markers from the string
real_ts_code = real_ts_code.replace("```tsx", "").replace("```", "").strip()

# Save the code to a .tsx file or print it
with open("output_code.tsx", "w") as file:
    file.write(real_ts_code)

print("TypeScript code written to output_code.tsx")

TypeScript code written to output_code.tsx
